In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pyspark

import datetime as dt

import tabula
import joblib

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 60)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import sklearn
from sklearn.preprocessing import StandardScaler, Binarizer, LabelBinarizer, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate \
                                    ,cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,recall_score,precision_score, f1_score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.metrics import roc_curve, auc

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE

In [43]:
files = ['raw/claims-2007-2009.xls', 'raw/claims-2010-2013.xls']

In [29]:
!ls

explore_raw_data.ipynb	reports
ideas.txt		simple_random_forest.ipynb
models			simple_random_forest_w_req_amount.ipynb
pdf_to_csvs.ipynb	tsa-wait-times-january-2006-december-2015.xls
raw


In [3]:
df_0206 = pd.read_excel('raw/claims-2002-2006.xls')
df_0709 = pd.read_excel('raw/claims-2007-2009.xls')
df_1013 = pd.read_excel('raw/claims-2010-2013.xls')
df_14 = pd.read_excel('raw/claims-2014.xls')
df_15 = pd.read_excel('raw/claims-data-2015-as-of-feb-9-2016-2.xlsx')

In [44]:
for i, file in enumerate(files):
    print(i, "::::", file)

0 :::: raw/claims-2007-2009.xls
1 :::: raw/claims-2010-2013.xls


In [81]:
# make dictionary of dfs
files = ['raw/claims-2007-2009.xls', 'raw/claims-2010-2013.xls'] # this should be all the dfs in production
d = {}
for i, file in enumerate(files):
    d[i] = pd.read_excel(file)  

# concat all dfs into one df
df = pd.concat(d, axis=0, ignore_index=True, sort=False)

# combine the Item and Item Category column into just Item Category
df['Item Category'].fillna(value=df['Item'], inplace=True)
df.drop(columns=['Item', 'Airport Name'], inplace=True)
df['Incident Date'] = pd.to_datetime(df['Incident Date'], errors='coerce')
df['Date Received'] = pd.to_datetime(df['Date Received'], errors='coerce')
df.dropna(subset=['Date Received', 'Incident Date'], inplace=True)


# This does something with duplicates. Don't run the code in production though, tweak with it once i have a master_df
df[(df.Status == 'Denied') | (df.Status == 'Approved') | (df.Status == 'Settled')][df.duplicated(subset=['Incident Date', 'Airport Code', 'Claim Amount', 'Status'], keep='first')].sort_values(by=['Claim Amount', 'Airport Code'], ascending=False)


/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [152]:
df.sample(15)

,Airline Name,Airport Code,Airport Name,Claim Amount,Claim Number,Claim Site,Claim Type,Close Amount,Date Received,Disposition,Incident Date,Item Category,Status
48005,American Airlines,RDU,Raleigh-Durham International,nan,2010011567599,Checked Baggage,Property Damage,0,2010-01-14,Deny,2010-01-11 00:00:00,Baggage/Cases/Purses; Travel Accessories,NaN
51604,Continental Airlines,LAX,Los Angeles International Airport,nan,2010060172225,Checked Baggage,Passenger Property Loss,0,2010-05-18,Deny,2010-05-03 09:50:00,Clothing,NaN
65219,Alaska Airlines,SAN,San Diego International,nan,2011081186019,Checked Baggage,Passenger Property Loss,0,2011-07-26,Deny,2011-07-16 06:30:00,Other,NaN
60697,American Airlines,DFW,Dallas-Fort Worth International Airport,nan,2011032881422,Checked Baggage,Passenger Property Loss,0,2011-03-01,Deny,2011-02-24 20:30:00,Other,NaN
88620,Delta Air Lines,ATL,Hartsfield-Jackson Atlanta International Airport,nan,2013112209618,Checkpoint,Property Damage,-,2013-11-21,-,2013-10-26 18:00:00,Home Decor,NaN
81512,Qantus Airways,LAX,Los Angeles International Airport,nan,2013030402347,Checked Baggage,Passenger Property Loss,0,2013-03-01,Deny,2013-02-22 00:00:00,-,NaN
38555,Delta Air Lines,HSV,Huntsville International Airport,62.640,2009041757729,Checked Baggage,Passenger Property Loss,0.000,2009-04-01,Deny,2009-02-22 00:00:00,"Cell Phones; MP3 Players-(iPods, etc)",Denied
84379,British Airways,F,Foreign airport,nan,2013080705748,Checked Baggage,Passenger Property Loss,0,2013-06-17,Deny,2013-05-26 17:00:00,Other,NaN
61070,Delta Air Lines,GEG,Spokane International,nan,2011040181725,Checked Baggage,Passenger Property Loss,0,2011-03-14,Deny,2011-03-06 22:20:00,Travel Accessories,NaN
42605,UAL,IAD,Washington Dulles International,400.000,2009072961908,Checked Baggage,Property Damage,0.000,2009-07-22,Deny,2009-07-05 00:00:00,Luggage (all types including footlockers),Denied


In [151]:
df[(df.Status == 'Denied') | (df.Status == 'Approved') | (df.Status == 'Settled')][df.duplicated(subset=['Incident Date', 'Airport Code', 'Claim Amount', 'Status'], keep='first')].sort_values(by=['Claim Amount', 'Airport Code'], ascending=False).count()

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Airline Name     104
Airport Code     111
Airport Name     111
Claim Amount     111
Claim Number     111
Claim Site       111
Claim Type       111
Close Amount     111
Date Received    111
Disposition      111
Incident Date    111
Item Category    107
Status           111
dtype: int64

In [110]:
df.Status.unique()

array(['Denied',
       'Insufficient, one of the following items required: sum certain, statement of fact, signature, location of incident, and date.',
       'Approved', 'Canceled', 'Closed as a contractor claim',
       'In litigation', 'Settled', 'In review',
       'Claim has been assigned for further investigation',
       'Pending response from claimant', nan], dtype=object)

In [108]:
itemlist = []
df['Item Category'].str.split(';').map(lambda x: itemlist.append(x) if x is not np.nan else 0)
flat_list = [item for sublist in itemlist for item in sublist]

flat_list2 = [item.strip() for item in flat_list]

set(flat_list2)

In [11]:
for i, df in enumerate([df_0206, df_0709, df_1013, df_14, df_15]):
    print(f'DF Index {i}')
    print(df.shape)
    print('-----')
    print(df.head(1))
    print('\n')

DF Index 0
(97231, 13)
-----
  Claim Number        Date Received        Incident Date Airport Code  \
0     0909802M  2002-01-04 00:00:00  2002-12-12 00:00:00          EWR   

                   Airport Name          Airline Name       Claim Type  \
0  Newark International Airport  Continental Airlines  Property Damage   

   Claim Site   Item  Claim Amount    Status  Close Amount      Disposition  
0  Checkpoint  Other       350.000  Approved       350.000  Approve in Full  


DF Index 1
(47912, 13)
-----
    Claim Number Date Received        Incident Date Airport Code  \
0  2007010518383    2007-01-01  2006-11-25 00:00:00          PHX   

                       Airport Name        Airline Name  \
0  Phoenix Sky Harbor International  Southwest Airlines   

                Claim Type       Claim Site                     Item  \
0  Passenger Property Loss  Checked Baggage  Automobile Parts; Other   

   Claim Amount  Status  Close Amount Disposition  
0       319.730  Denied         0.0